In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib tk

#********************************
#    Variable Initializations   #
#********************************
# The lattice grid dimensions
NX = 15
NY = 10
# The relaxation time 1/T
omega = 1.0
# Weights
w_i = [4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36]
# Probability Density
f_ijk = np.einsum('ijk, i -> ijk', np.ones([9,NY,NX]), w_i)
# Velocity vector for different positions
c_ij = np.array([[0, 1, 0, -1, 0, 1, -1, -1, 1],
                 [0, 0, 1, 0, -1, 1, 1, -1, -1]])

#********************************
#       Common Functions        #
#********************************

# Function to calculate density
def calc_density(prob_density_ijk):
    return np.einsum('ijk->jk', prob_density_ijk)
    
# Function to calculate the velocity field at each grid point
def calc_vel_field(prob_density_ijk):
    rho = calc_density(prob_density_ijk)
    return np.einsum('ji, ikl -> jkl', c_ij, prob_density_ijk)/rho

# Streaming function
def streaming(prob_density_ijk):
    for i in range(1,9):
        prob_density_ijk[i] = np.roll(prob_density_ijk[i,:,:], shift=c_ij.T[i], axis=(0,1))
    return prob_density_ijk

# Function to plot the velocity field
def plot_vel_field(vel_field_ijk, iteration, fig, ax):
    if (iteration == 0):
        ax.set_title('The Streamplot of the velocity field')
    Y, X = np.mgrid[0:NY,0:NX]
    stream = ax.streamplot(X, Y, vel_field_ijk[0], vel_field_ijk[1], density=1, color='red')
    return stream

# Function to plot the density
def plot_density(density_ij):
    fig, ax = plt.subplots()
    denseplot = ax.pcolormesh(np.arange(density_ij.shape[1]), np.arange(density_ij.shape[0]), density_ij, shading='auto')
    fig.colorbar(denseplot)
#     ax.set_title('Density across all grid points')

# Function to find the equilibrium distribution function
def calc_feq(density_ij, velocity_ijk):
    eq_prob_density_ijk = np.ones([9,NY,NX])
    # Loop through all the channels
    for i in range(0,9):
        cu_ij = np.einsum('n, njk -> jk', c_ij.T[i], velocity_ijk)
        cu2_ij = np.einsum('ij, ij -> ij', cu_ij, cu_ij)
        u2_ij = np.einsum('ijk, ijk -> jk', u_ijk, u_ijk)
        eq_prob_density_ijk[i] = w_i[i] * density_ij * (1 + 3 * cu_ij + (9 / 2) * cu2_ij - (3 / 2) * u2_ij)
    return eq_prob_density_ijk

# Collision function
def collision(prob_density_ijk, eq_prob_density_ijk):
    for i in range(0,9):
        prob_density_ijk[i] = prob_density_ijk[i] + omega * (eq_prob_density_ijk[i] - prob_density_ijk[i])
    return prob_density_ijk

In [2]:
# Initialise the density and velocity to test the streaming and collision functions
rho_ij = np.ones([NY, NX])
rho_ij[2,2] = 0.99
u_ijk = np.zeros([2, NY, NX])

# Initialize probability distribution
feq_ijk = calc_feq(rho_ij, u_ijk)
f_ijk = collision(f_ijk, feq_ijk)
f_ijk = feq_ijk.copy()

In [3]:
#**********************************
#      Animated Stream Plot       #
#**********************************
fig, ax = plt.subplots()

def stabilize():
    global f_ijk, feq_ijk, rho_ij, u_ijk
    # Streaming
    f_ijk = streaming(f_ijk)
    # Calculating the density and velocity
    rho_ij = calc_density(f_ijk)
    u_ijk = calc_vel_field(f_ijk)
    # Calculate the equilibrium distribution function
    feq_ijk = calc_feq(rho_ij, u_ijk)
    # Calculating the new probability distribution
    f_ijk = collision(f_ijk, feq_ijk)

def init():
    animate(0)

def animate(iter):
    ax.collections = [] # clear lines streamplot
    ax.patches = [] # clear arrowheads streamplot
    stabilize()
    plot_vel_field(u_ijk, iter, fig, ax)

anim = FuncAnimation(fig, animate, init_func=init, frames=800, interval=20, blit=False, repeat=False)
# anim.save('./animation.gif', writer='Pillow', fps=60)

In [ ]:
#*******************************
#   Test - Checkered density   #
#*******************************

rho_ij = np.ones([NY, NX])
rho_ij[0::2,1::2] = rho_ij[0::2,1::2] + 0.01
rho_ij[0::2,0::2] = rho_ij[0::2,0::2] - 0.01
rho_ij[1::2,1::2] = rho_ij[1::2,1::2] - 0.01
rho_ij[1::2,0::2] = rho_ij[1::2,0::2] + 0.01
u_ijk = np.zeros([2, NY, NX])

# Initialize probability distribution
feq_ijk = calc_feq(rho_ij, u_ijk)
f_ijk = feq_ijk.copy()

# Changing velocities at the 1st and 4th channels.
# f_ijk[1,0,:] = f_ijk[1,0,:] + 0.01
# f_ijk[1,1,:] = f_ijk[1,1,:] - 0.01
# f_ijk[4,0,:] = f_ijk[4,0,:] + 0.01
# f_ijk[4,1,:] = f_ijk[4,1,:] - 0.01

# Stream and plot
fig, ax = plt.subplots()
anim = FuncAnimation(fig, animate, init_func=init, frames=800, interval=20, blit=False, repeat=False)